In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import float64

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5"]
column_labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5", "Ball Delta", "Ball Y Delta", "LS Delta", "LS Y Delta",
    "Ball 1 Repeat Last 5", "Ball 2 Repeat Last 5", "Ball 3 Repeat Last 5", "Ball 4 Repeat Last 5", "Ball 5 Repeat Last 5", "Lucky Star 1 Repeat Last 5", "Lucky Star 2 Repeat Last 5",
    "Ball 1 Repeat Last 10", "Ball 2 Repeat Last 10", "Ball 3 Repeat Last 10", "Ball 4 Repeat Last 10", "Ball 5 Repeat Last 10", "Lucky Star 1 Repeat Last 10", "Lucky Star 2 Repeat Last 10",
    "Ball 1 Repeat Last 20", "Ball 2 Repeat Last 20", "Ball 3 Repeat Last 20", "Ball 4 Repeat Last 20", "Ball 5 Repeat Last 20", "Lucky Star 1 Repeat Last 20", "Lucky Star 2 Repeat Last 20",
     "Digit 0 Freq", "Digit 1 Freq", "Digit 2 Freq", "Digit 3 Freq", "Digit 4 Freq", "Digit 5 Freq", "Digit 6 Freq", "Digit 7 Freq", "Digit 8 Freq", "Digit 9 Freq"
]
column_indices = {name: i for i, name in enumerate(column_labels)}
digit_indices = {}


df1 = pd.read_excel("lottery-numbers-tracker.xlsx", sheet_name="EuroMillions", header=0)
df2 = pd.read_excel("lottery-numbers-tracker.xlsx", sheet_name="EuroMillions digit analysis", header=0)

pd.concat([df1, df2], axis=1)[column_labels].to_csv("euromillions-dataset.csv", index=None)

Num GPUs Available:  0


In [2]:
df = pd.read_csv('euromillions-dataset.csv')
df

,Lucky Star 1,Lucky Star 2,Ball 1,Ball 2,Ball 3,Ball 4,Ball 5,Ball Delta,Ball Y Delta,LS Delta,...,Digit 0 Freq,Digit 1 Freq,Digit 2 Freq,Digit 3 Freq,Digit 4 Freq,Digit 5 Freq,Digit 6 Freq,Digit 7 Freq,Digit 8 Freq,Digit 9 Freq
0,3,8,14,28,34,46,49,4.905354,5.223026,5,...,0,1,1,2,4,0,1,0,2,1
1,1,7,3,14,15,48,49,8.703448,8.687923,6,...,0,3,0,1,3,1,0,1,1,1
2,5,7,27,39,41,45,50,3.436932,5.184593,2,...,1,1,1,1,2,3,0,2,0,1
3,4,12,9,24,30,46,49,5.733672,1.296148,8,...,1,1,2,1,4,0,1,0,0,2
4,1,9,10,21,30,42,45,4.710361,5.332917,8,...,2,3,2,1,2,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,6,9,1,19,36,38,49,6.791539,2.814249,3,...,0,2,0,2,1,0,2,0,1,3
326,9,10,10,25,29,34,45,4.918079,3.143247,1,...,2,2,2,1,2,2,0,0,0,2
327,2,4,6,16,18,39,47,6.169481,3.831449,2,...,0,2,1,1,2,0,2,1,1,1
328,7,12,5,14,35,36,39,5.766281,4.422669,5,...,0,2,1,3,1,2,1,1,0,1


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# --- 1. Data Preparation ---

def create_sequences(data, seq_length, pred_cols):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)].values[::-1]
        y = data.iloc[i + seq_length, :pred_cols].values
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# --- 2. Load and Preprocess Data ---

# Parameters
seq_length = 7  # Number of past rows to look at
pred_cols = 42  # Number of columns to predict
ball_cols = 7 # Numbers of columns that represent ball numbers
all_cols = 42
random_seed_length = 7 # Length of random seed

In [4]:
max_values = np.concatenate(([12, 12, 50, 50, 50, 50, 50], np.zeros(all_cols - ball_cols)))
min_values = np.concatenate(([1, 1, 1, 1, 1, 1, 1], np.zeros(all_cols - ball_cols)))

# Scale the data
scaler = MinMaxScaler()
scaled_df = df[::-1]
scaled_df = pd.concat([scaled_df, pd.DataFrame([max_values], columns=df.columns)], ignore_index=True)
scaled_df = pd.concat([scaled_df, pd.DataFrame([min_values], columns=df.columns)], ignore_index=True)
scaled_df.shape

(332, 42)

In [5]:
# Scale the data
scaled_df = pd.DataFrame(scaler.fit_transform(scaled_df), columns=df.columns)
scaled_df = scaled_df.iloc[:-2]

# Create sequences
X, y = create_sequences(scaled_df, seq_length, pred_cols)

# Split into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

# Input for the sequence
sequence_input = tf.keras.layers.Input(shape=(seq_length, all_cols))

# Input for the random seed
random_seed_input = tf.keras.layers.Input(shape=(random_seed_length,))

scaled_df.shape

(330, 42)

In [6]:
scaler.inverse_transform([np.concatenate((y[-1], np.zeros(all_cols - pred_cols)))])

array([[ 3.        ,  8.        , 14.        , 28.        , 34.        ,
        46.        , 49.        ,  4.90535422,  5.22302594,  5.        ,
         1.58113883,  1.        ,  0.        ,  0.        ,  1.        ,
         2.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  2.        ,  3.        ,  1.        ,  2.        ,
         3.        ,  0.        ,  0.        ,  4.        ,  4.        ,
         2.        ,  3.        ,  0.        ,  1.        ,  1.        ,
         2.        ,  4.        ,  0.        ,  1.        ,  0.        ,
         2.        ,  1.        ]])

In [7]:
scaler.inverse_transform(X[-1])

array([[ 1.        ,  7.        ,  3.        , 14.        , 15.        ,
        48.        , 49.        ,  8.70344759,  8.68792265,  6.        ,
         2.82842712,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  0.        ,
         1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         4.        ,  2.        ,  1.        ,  1.        ,  4.        ,
         3.        ,  4.        ,  0.        ,  3.        ,  0.        ,
         1.        ,  3.        ,  1.        ,  0.        ,  1.        ,
         1.        ,  1.        ],
       [ 5.        ,  7.        , 27.        , 39.        , 41.        ,
        45.        , 50.        ,  3.43693177,  5.18459256,  2.        ,
         3.60555128,  0.        ,  0.        ,  0.        ,  1.        ,
         2.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  2.        ,  2.        ,  1.        ,
         1.     

In [8]:
# for testing sequences
# df = df[::-1]
# print(y[-1])
# print(df.iloc[-1].values[:7])

In [9]:
# --- 3. Model Definition ---

# LSTM layer
lstm1_1 = tf.keras.layers.LSTM(128, return_sequences=False)(sequence_input)

dense_1 = tf.keras.layers.Dense(128, activation='sigmoid')(lstm1_1)

dense_2 = tf.keras.layers.Dense(35, activation='sigmoid')(dense_1)

concat = tf.keras.layers.Concatenate()([random_seed_input, dense_2])

output = tf.keras.layers.Dense(pred_cols)(concat)

# Create the model with two inputs
lstm_model = tf.keras.models.Model(inputs=[sequence_input, random_seed_input], outputs=output)

In [10]:

# --- 4. Loss Function with Emphasis ---

def custom_loss(y_true, y_pred, random_target):
    # Calculate loss
    squared_error = tf.square(y_true - y_pred)
    weighted_mse_loss = tf.reduce_mean(squared_error)

    # MSE between prediction and random target
    mse_loss_random = tf.reduce_mean(tf.square(y_pred[:, :ball_cols] - random_target))

    # Combine the two losses (you can adjust the weights)
    return tf.abs(0.2 * weighted_mse_loss - mse_loss_random)

def generate_random_target(batch_size):
    targets = []
    for _ in range(batch_size):
        lucky_target = []
        target = []
        while len(lucky_target) < 2:
            num = np.random.randint(1, 13)
            if num not in lucky_target:
                lucky_target.append(num)
        while len(target) < 5:
            num = np.random.randint(1, 51)
            if num not in target:
                target.append(num)
        lucky_target = np.array(lucky_target) / 12
        target = np.array(target) / 50
        targets.append(np.concatenate((lucky_target, target)))
    return np.array(targets)

# --- 5. Compile the Model ---

lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   loss=custom_loss)  # Use the custom weighted loss


In [11]:
print(X_train.shape)

print(scaler.data_min_)

print(scaler.data_max_)

(290, 7, 42)
[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[12.         12.         50.         50.         50.         50.
 50.          8.70344759 11.56719499 11.          9.05538514  2.
  3.          4.          3.          3.          3.          4.
  5.          5.          5.          6.          5.          5.
  6.          6.          6.          7.          6.          7.
  9.         10.          4.          7.          6.          6.
  6.          3.          4.          3.          4.          3.        ]


In [ ]:
# --- 6. Train the Model ---

# Custom training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

epochs = 300
batch_size = 16

for epoch in range(epochs):
    epoch_loss = 0
    for i in range(0, X_train.shape[0], batch_size):
        # Get batch data

        X_batch = X_train[i:(i + batch_size)]
        y_batch = y_train[i:(i + batch_size)]
        random_seeds_batch = generate_random_target(X_batch.shape[0])

        with tf.GradientTape() as tape:
            # Get predictions
            predictions = lstm_model([X_batch, random_seeds_batch], training=True)

            # Combine the two losses (you can adjust the weights)
            loss = custom_loss(y_batch, predictions, random_seeds_batch)

        # Compute and apply gradients
        gradients = tape.gradient(loss, lstm_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, lstm_model.trainable_variables))

        epoch_loss += loss

    # Print epoch loss
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / (X_train.shape[0] // batch_size)}")



Epoch 1/300, Loss: 0.4083667993545532
Epoch 2/300, Loss: 0.030677011236548424


In [37]:
# --- 8. Evaluate the Model ---

val_loss = 0
for i in range(0, X_val.shape[0], batch_size):
    X_batch = X_val[i:i + batch_size]
    y_batch = y_val[i:i + batch_size]
    random_target_batch = generate_random_target(X_batch.shape[0])

    predictions = lstm_model([X_batch, random_target_batch], training=False)
    loss = custom_loss(y_batch, predictions, random_target_batch)
    val_loss += loss

print(f"Validation Loss: {val_loss / (X_val.shape[0] // batch_size)}")

Validation Loss: 0.005613568238914013


In [40]:
# --- 9. Make Predictions ---

# Generate a random seed for prediction
prediction_random_seed = generate_random_target(1)

# Get the last sequence from X_val and reshape it for prediction
last_sequence = X[-1].reshape(1, seq_length, all_cols)

# Make a prediction using the last sequence and the random seed
prediction = lstm_model.predict([last_sequence, prediction_random_seed])

# Inverse transform to get predictions in the original scale
prediction = scaler.inverse_transform(np.concatenate((prediction, np.zeros((prediction.shape[0], all_cols - pred_cols))), axis=1))[:, :pred_cols]

prediction

1/1 [==============================] - 0s 12ms/step


array([[ 5.51711345e+00,  7.65800440e+00,  3.43354067e+01,
         2.08290841e+01,  3.54952317e+01,  2.14436878e+01,
         1.28126178e+01, -3.33825307e-01,  1.08149785e+01,
        -1.06480868e+00,  1.47899747e+00,  2.36641131e-02,
         1.49769330e+00,  3.59588289e+00,  9.60321486e-01,
         8.71640682e-01,  6.53401807e-01, -4.17626202e-01,
         2.81904846e+00,  1.90433279e+00, -8.51191282e-01,
         3.56696248e+00,  1.48188084e+00,  2.15056196e+00,
         2.72825074e+00,  6.03981704e-01,  2.57563090e+00,
        -2.09501749e+00,  7.59761542e-01,  5.73149049e+00,
        -1.65711594e+00,  7.23529041e+00,  8.48886132e-01,
         5.06474417e+00,  7.64598906e-01,  4.89528000e-01,
         1.75723422e+00,  1.24443206e+00, -1.74341202e+00,
         3.68262291e-01,  4.76923943e-01,  8.97467226e-01]])

In [16]:
print(scaler.inverse_transform(X[-1])[:, :pred_cols]);

[[ 5. 12.  9. 17. 18. 21. 50.]
 [ 8. 11.  4.  9. 35. 46. 50.]
 [ 2.  9.  8. 10. 33. 35. 49.]
 [ 7. 11. 13. 36. 37. 42. 47.]
 [ 3.  8.  6. 10. 27. 32. 48.]
 [ 2.  9. 15. 20. 29. 35. 42.]
 [ 5.  9.  7. 16. 31. 43. 46.]]
